In [9]:
using DifferentialEquations, Serialization
include("ode_problem.jl")
include("calculate_experimental_outputs.jl")
include("make_predictions.jl");

Specify Run of Interest

In [10]:
#if running from command line, can set this globally and run for different cases
#run = "201";

Create ODE Problems for each initial condition

In [11]:
#next input we need is an ode problem
ligand_dosages = [20.0, 2.0, 0.2] #all possible doses
n_doses = length(ligand_dosages)
odeproblems = []
for egf in ligand_dosages
    odesys, u0, tspan, p = return_ode_problem_default_inputs(egf)
    prob = DifferentialEquations.ODEProblem(odesys, u0, tspan, p) #note p will be overwritten in first run of mcmc algorithm
    push!(odeproblems, prob)
end

Load ODE Solver Inputs

In [12]:
#need ode solver inputs, which are consistent for all simulations
odesolver_inputs = return_ode_problem_solver_default_inputs()
save_at_finegrain = return_finegrain_saveat()
odesolver_inputs_finegrain = copy(odesolver_inputs)
odesolver_inputs_finegrain["saveat"] = save_at_finegrain;

Define Type of ODE Solution

In [13]:
#get type of ODEProblem solution
pred = DifferentialEquations.solve(odeproblems[1], odesolver_inputs["solver"], abstol=odesolver_inputs["abstol"], reltol=odesolver_inputs["reltol"], saveat=odesolver_inputs["saveat"])
ode_sol_type = typeof(pred);

Load posterior samples and convert from log10 scale

In [14]:
parameters = 10.0.^deserialize("outputs/300_$(run)_posterior_samples_thinned.jls")
n_parameters, n_samples = size(parameters);

Simulate using Posterior Samples

In [15]:
predicted = Matrix{ode_sol_type}(undef, n_doses, n_samples)
predicted_finegrain = Matrix{ode_sol_type}(undef, n_doses, n_samples)
for i in 1:n_doses
    for j in 1:n_samples
        predicted[i,j] = make_prediction(odeproblems[i], odesolver_inputs, parameters[:,j])
        predicted_finegrain[i,j] = make_prediction(odeproblems[i], odesolver_inputs_finegrain, parameters[:,j])
    end
end
serialize("outputs/500_$(run)_predictions.jls", predicted)
serialize("outputs/500_$(run)_predictions_finegrain.jls", predicted_finegrain)

Calculate Experiemental Quantities per Dose

In [ ]:
dose_names = ["20","2","02"]
for i in 1:n_doses
    experimental_quantities = Array{Dict}(undef, n_samples)
    experimental_quantities_finegrain = Array{Dict}(undef, n_samples)
    for j in 1:n_samples
        experimental_quantities[j] = calculate_all_quantities(predicted[i,j])
        experimental_quantities_finegrain[j] = calculate_all_quantities(predicted_finegrain[i,j])
    end
    exp_keys = keys(experimental_quantities[1])
    for k in exp_keys
        one_quantity = [experimental_quantities[j][k] for j in 1:n_samples]
        one_quantity_finegrain = [experimental_quantities_finegrain[j][k] for j in 1:n_samples]
        serialize("outputs/500_$(run)_predictions_$(k)_$(dose_names[i]).jls", one_quantity)
        serialize("outputs/500_$(run)_finegrain_predictions_$(k)_$(dose_names[i]).jls", one_quantity_finegrain)
    end
end